# Prepare Open AI client object

In [1]:
from openai import OpenAI

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
client=OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [4]:
client

# Prepare documents for information retrieval

In [5]:
import minsearch

In [6]:
import json

In [7]:
with open('documents.json', 'rt') as f_in:
    docs_raw=json.load(f_in)

In [8]:
documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [9]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course']
)
index.fit(documents)

In [27]:
query ='the course has already started, can I still enroll?'

In [24]:
def search(query):
    
    boost={'question': 3.0, 'section': 0.5 } # By default, all parameters have value of 1.
    #Increasing or decreasing this value will change its relevance during the search process.

    results = index.search(
        query=query,
        boost_dict=boost,
        num_results=5,
        filter_dict={'course': 'data-engineering-zoomcamp'}
    )
    return results

In [28]:
search_results = search(query)

# Generate answers from OpenAI GPT

In [38]:
def build_prompt(query, search_results):
    
    prompt_template="""
    You are a course teaching assistant. Answer the QUESTION based on the CONTEXT.
    Use only the facts of the CONTEXT in the answer. 
    If the CONTEXT doesn't contain the answer, output NONE.

    QUESTION: {question}

    CONTEXT: {context}

    """.strip()
    
    context=''

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer:{doc['text']}\n\n"
        
    prompt = prompt_template.format(question=query, context=context).strip()
        
    return prompt

In [39]:
def llm(prompt):
    
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

Test generation without context

In [14]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role':'user', 'content':q}]
)

In [15]:
response.choices[0].message.content

'Yes, whether you can still enroll in a course after it has started typically depends on the specific policies of the institution or platform offering the course. Here are some steps you can take:\n\n1. **Check the Course Policy**: Look at the enrollment guidelines provided by the institution or on the course website. Some courses have deadlines, while others may allow late enrollment.\n\n2. **Contact the Instructor or Administrator**: Reach out directly to the course instructor or the administration office. Explain your situation and ask if late enrollment is possible.\n\n3. **Catch Up on Missed Work**: If you are allowed to enroll, you may need to catch up on any missed lectures, assignments, or readings. Ask for any supplementary materials or advice on how to get up to speed.\n\n4. **Use Online Forums or Classmates**: If the course has an online forum or discussion board, use it to ask questions and get help. Connecting with classmates can also provide support and resources to help 

Test generation with context

In [40]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [41]:
answer = rag(query)

In [42]:
answer

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [43]:
rag('how can i make pancakes?')

'NONE'

# RAG with Elastic Search

In [44]:
from elasticsearch import Elasticsearch

In [46]:
es_client=Elasticsearch('http://localhost:9200')

In [47]:
#es_client.info()

ObjectApiResponse({'name': 'b359dd4497f3', 'cluster_name': 'docker-cluster', 'cluster_uuid': '_7h4zc2MQCKCk6sS0pWVbw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [48]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

#Create an index in elastic search (equivelant to a table in sql database)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [49]:
for doc in documents:
    
    es_client.index(index=index_name, document=doc)

In [51]:
query= 'i just found out about this course, can i still join?'

In [52]:
def elastic_search(query):
    
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        #a question is 3 times more important than the text or section for the search results
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": { #limit questions about only one part of the documents available
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    search_results = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in search_results['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [53]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [54]:
def elastic_rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [55]:
elastic_rag(query)

'Yes, you can still join the course even after the start date. You are eligible to submit the homework assignments, but keep in mind that there will be deadlines for turning in the final projects. So make sure not to leave everything for the last minute.'

In [56]:
rag(query)

"Yes, you can still join the course. Even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects, so don't leave everything for the last minute."

# Homework

In [58]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [59]:
query="How do I execute a command in a running docker container?"

In [ ]:
def elastic_search(query):
    
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
             #   "filter": { #limit questions about only one part of the documents available
             #       "term": {
              #          "course": "data-engineering-zoomcamp"
              #      }
                }
            }
        }
    }

    search_results = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in search_results['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs